# 1. Convolutional Neurak Networks (CNNs/ConvNet)
ConvNet architectures make the explicit assumption that the inputs are images, which allows us to encode certain properties into the architecture. These then make the forward function more efficient to implement and vastly reduce the amount of parameters in the network.


  

## 1.1 Architecture Overview

## 1.2 Layers used to build ConvNets


### 1.2.1 Convolutional Layer
#### Overview and intuition without brain stuff
- Every filter is small spatially (along with width and height), but extends through the full depth of the input voloum.
- Intuitively, the network will learn filters that activate when they see some type of visual feature.
#### The brain view
- Every entry in the 3D output volume can also be interpreted as an output of a neuron that looks at only a small region in the input and shares parameters with all neurons to the left and right spatially.
#### Local Connectivity
- We will connect each neuron to only a local region of the input volume. The spatial extent of this connectivity is a hyperparameter called the **receptive field** of the neuron (equivalently this is the **filter size**).
#### Spatial arrangement
- The **depth** of the output volume is a hyperparameter: it corresponds to the number of filters we would like to use, each learning to look for something different in the input.
- We must specify the stride with which we slide the filter. When the stride is 1 then we move the filters one pixel at a time. When the stride is 2 (or uncommonly 3 or more, though this is rare in practice) then the filters jump 2 pixels at a time as we slide them around. This will produce smaller output volumes spatially.
##### Use of zero-padding
- The nice feature of zero padding is that it will allow us to control the spatial size of the output volumes (most commonly as we’ll see soon we will use it to exactly preserve the spatial size of the input volume so the input and output width and height are the same). 
- $Output size = \frac{W - F + 2P}{S} + 1$. $W$: input size, $F$: receptive field, $S$: stride, $P$: amount of zero padding.
- Setting $P = \frac{F - 1}{2}$ when $S=1$ ensures that the input and output will have the same size spatially.
##### Constraints on strides
- ... integer
##### Real-world example
- ... 
#### Parameter Sharing
- That if one feature is useful to compute at some spatial position $(x,y)$, then it should also be useful to compute at a different position $(x_2,y_2)$
- Note that sometimes the parameter sharing assumption may not make sense. This is especially the case when the input images to a ConvNet have some specific centered structure, where we should expect, for example, that completely different features should be learned on one side of the image than another.
#### Numpy examples
#### Implementation as Matrix Multiplication
**1.** The local regions in the input image are stretched out into columns in an operation commonly called im2col. For example, if the input is [227x227x3] and it is to be convolved with 11x11x3 filters at stride 4, then we would take [11x11x3] blocks of pixels in the input and stretch each block into a column vector of size 11*11*3 = 363. Iterating this process in the input at stride of 4 gives (227-11)/4+1 = 55 locations along both width and height, leading to an output matrix X_col of im2col of size [363 x 3025], where every column is a stretched out receptive field and there are 55*55 = 3025 of them in total. Note that since the receptive fields overlap, every number in the input volume may be duplicated in multiple distinct columns.
**2.** The weights of the CONV layer are similarly stretched out into rows. For example, if there are 96 filters of size [11x11x3] this would give a matrix W_row of size [96 x 363].
**3.** The result of a convolution is now equivalent to performing one large matrix multiply np.dot(W_row, X_col), which evaluates the dot product between every filter and every receptive field location. In our example, the output of this operation would be [96 x 3025], giving the output of the dot product of each filter at each location.
**4.** The result must finally be reshaped back to its proper output dimension [55x55x96].

#### Backpropagation
#### Dilated convolutions
- if you stack two 3x3 CONV layers on top of each other then you can convince yourself that the neurons on the 2nd layer are a function of a 5x5 patch of the input (we would say that the effective receptive field of these neurons is 5x5). If we use dilated convolutions then this effective receptive field would grow much quicker.


### 1.2.2 Pooling Layer
- It is common to periodically insert a Pooling layer in-between successive Conv layers in a ConvNet architecture. Its function is to progressively reduce the spatial size of the representation to reduce the amount of parameters and computation in the network, and hence to also control **overfitting**
- Input: $W_1, H_1, D_1$, spatial extent $F$, stride $S$, Output: $W_2 = \frac{W_1 - F}{S} + 1$, $H_2 = \frac{H_1 - F}{S} + 1$, $D_2 = D_1$.
#### General pooling
- Average pooling
- L2-norm pooling.
#### Backpropagation
- During the forward pass of a pooling layer it is common to keep track of the index of the max activation (sometimes also called the switches) so that gradient routing is efficient during backpropagation.
#### Getting rid of pooling
- Discarding pooling layers has also been found to be important in training good generative models, such as variational autoencoders (VAEs) or generative adversarial networks (GANs).

### 1.2.3 Normalizaion Layer 
- Many types of normalization layers have been proposed for use in ConvNet architectures, sometimes with the intentions of implementing inhibition schemes observed in the biological brain. However, these layers have since fallen out of favor because in practice their contribution has been shown to be minimal, if any

### 1.2.4 Fully-connected Layer 


### 1.2.5 Converting FC layers to CONV layers
- It is worth noting that the only difference between FC and CONV layers is that the neurons in the CONV layer are connected only to a local region in the input, and that many of the neurons in a CONV volume share parameters.
- The neurons in both layers still compute dot products, so their functional form is identical.
- For any CONV layer there is an FC layer that implements the same forward function. The weight matrix would be a large matrix that is mostly zero except for at certain blocks (due to local connectivity) where the weights in many of the blocks are equal (due to parameter sharing).
#### FC -> CONV conversion
- If 224x224 image gives a volume of size [7x7x512] - i.e. a reduction by 32, then forwarding an image of size 384x384 through the converted architecture would give the equivalent volume in size [12x12x512], since 384/32 = 12. Following through with the next 3 CONV layers that we just converted from FC layers would now give the final volume of size [6x6x1000], since (12 - 7)/1 + 1 = 6. Note that instead of a single vector of class scores of size [1x1x1000], we’re now getting an entire 6x6 array of class scores across the 384x384 image.
- **Forwarding the converted ConvNet a single time is much more efficient than iterating the original ConvNet over all those 36 locations**, since the 36 evaluations share computation. This trick is often used in practice to get better performance, where for example, it is common to resize an image to make it bigger, use a converted ConvNet to evaluate the class scores at many spatial positions and then average the class scores.

## 1.3 ConvNet Architecture


### 1.3.1 Layer Patterns
- The most common form of a ConvNet architecture stacks a few CONV-RELU layers, follows them with POOL layers, and repeats this pattern until the image has been merged spatially to a small size. At some point, it is common to transition to fully-connected layers. The last fully-connected layer holds the output, such as the class scores.
- $INPUT -> [[CONV -> RELU]*N -> POOL?]*M -> [FC -> RELU]*K -> FC$, $POOL?$ is optional pooling layer. 
- **Prefer a stack of small filter CONV to one large receptive field CONV layer.**
- In practice: use whatever works best on ImageNet

### 1.3.2 Layer Sizing Patterns
- The input layer (that contains the image) should be divisible by 2 many times. Common numbers include 32 (e.g. CIFAR-10), 64, 96 (e.g. STL-10), or 224 (e.g. common ImageNet ConvNets), 384, and 512.
- The conv layers should be using small filters (e.g. 3x3 or at most 5x5), using a stride of S=1, and crucially, padding the input volume with zeros in such way that the conv layer does not alter the spatial dimensions of the input.
- The pool layers are in charge of downsampling the spatial dimensions of the input. The most common setting is to use max-pooling with 2x2 receptive fields (i.e. F=2), and with a stride of 2 (i.e. S=2)
- Reducing sizing headaches. The scheme presented above is pleasing because all the CONV layers preserve the spatial size of their input, while the POOL layers alone are in charge of down-sampling the volumes spatially.
- Why use stride of 1 in CONV? Smaller strides work better in practice. Additionally, as already mentioned stride 1 allows us to leave all spatial down-sampling to the POOL layers, with the CONV layers only transforming the input volume depth-wise.
- Why use padding? In addition to the aforementioned benefit of keeping the spatial sizes constant after CONV, doing this actually improves performance. If the CONV layers were to not zero-pad the inputs and only perform valid convolutions, then the size of the volumes would reduce by a small amount after each CONV, and the information at the borders would be “washed away” too quickly


### 1.3.3 Case studies
- LeNet: http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf
- AlexNet: http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks
- ZF Net: https://arxiv.org/abs/1311.2901
- GoogLeNet: https://arxiv.org/abs/1409.4842
- VGGNet: http://www.robots.ox.ac.uk/~vgg/research/very_deep/
- ResNet: https://arxiv.org/abs/1512.03385
- Batch normalization: https://arxiv.org/abs/1502.03167

### 1.3.4 Computational Considerations
- The memory to store the parameter vector alone must usually be multiplied by a factor of at least 3 or so.
- Once you have a rough estimate of the total number of values (for activations, gradients, and misc), the number should be converted to size in GB. Take the number of values, multiply by 4 to get the raw number of bytes (since every floating point is 4 bytes, or maybe by 8 for double precision), and then divide by 1024 multiple times to get the amount of memory in KB, MB, and finally GB. If your network doesn’t fit, a common heuristic to “make it fit” is to decrease the batch size, since most of the memory is usually consumed by the activations.

# 2. Neural Network

## 2.1 Quick intro


## 2.2 Modeling one neuron


### 2.2.1 Biological motivation and connections


### 2.2.2 Single neuron as a linear classifer
**Binary Softmax classifier**
We can interpret $\sigma (\sum_i w_ix_i+b)$ to be the probability of one of the classes $P(y_i=1∣x_i;w)$. The probability of the other class would be $P(y_i=0∣x_i;w)=1−P(y_i=1∣x_i;w)$, since they must sum to one.

**Binary SVM classifier**
We could attach a max-margin hinge loss to the output of the neuron and train it to become a binary Support Vector Machine.
**Regularization interpretation**
The regularization loss in both SVM/Softmax cases could in this biological view be interpreted as gradual forgetting, since it would have the effect of driving all synaptic weights w towards zero after every parameter update.

### 2.2.3 Commonly used activation functions
Every activation function (or non-linearity) takes a single number and performs a certain fixed mathematical operation on it.
- **Sigmoid**: The sigmoid non-linearity has the mathematical form $\sigma (x)=\frac{1}{1+e^{−x}}$. In practice, the sigmoid non-linearity has recently fallen out of favor and it is rarely ever used. It has two major drawbacks:
-   *Sigmoids saturate and kill gradients*
-   *Sigmoid outputs are not zero-centered.*
- **Tanh**: The tanh neuron is simply a scaled sigmoid neuron, in particular the following holds: $tanh(x)=2\sigma (2x)−1$. Like the sigmoid neuron, its activations saturate, but unlike the sigmoid neuron its output is zero-centered. Therefore, in practice the tanh non-linearity is always preferred to the sigmoid nonlinearity.
- **ReLu**: $f(x) = max(0,x)$. 
Pros:
+ It was found to greatly accelerate the convergence of stochastic gradient descent compared to the sigmoid/tanh functions.
+ Simple
Cons: 
+ A large gradient flowing through a ReLU neuron could cause the weights to update in such a way that the neuron will never activate on any datapoint again. If this happens, then the gradient flowing through the unit will forever be zero from that point on. That is, the ReLU units can irreversibly die during training since they can get knocked off the data manifold. 
- **Leaky ReLU**: $f(x)=1(x<0)(\alpha x)+1(x>=0)(x)$. -> fix "dying ReLU"
- **Maxout**: $max(w^T_1x+b_1,w^T_2x+b_2)$. -> double parameters. 

Conclude: Use ReLU. If concern about dead unit, use Leaky ReLU or Maxout. 

# 2.3 Neural Network architecture


## 2.3.1 Layer-wise organization
**Naming convention**: When we say N-layer neural network, we do not count the input layer.

**Output layer**: Unlike all layers in a Neural Network, the output layer neurons most commonly do not have an activation function. This is because the last output layer is usually taken to represent the class scores, which are arbitrary real-valued numbers, or some kind of real-valued target.

**Sizing neural networks**: The two metrics that people commonly use to measure the size of neural networks are the number of neurons, or more commonly the number of parameters.

## 2.3.2 Example feed-forward computation


## 2.3.3 Representation power
- Neural Networks work well in practice because they compactly express nice, smooth functions that fit well with the statistical properties of data we encounter in practice, and are also easy to learn using our optimization algorithms (e.g. gradient descent). Similarly, the fact that deeper networks (with multiple hidden layers) can work better than a single-hidden-layer networks is an empirical observation, despite the fact that **their representational power is equal**.


## 2.3.4 Setting number of layers and their sizes.
- The subtle reason behind this is that smaller networks are harder to train with local methods such as Gradient Descent: It’s clear that their loss functions have relatively few local minima, but it turns out that many of these minima are easier to converge to, and that they are bad (i.e. with high loss). Conversely, bigger neural networks contain significantly more local minima, but these minima turn out to be much better in terms of their actual loss. Since Neural Networks are non-convex, it is hard to study these properties mathematically, but some attempts to understand these objective functions have been made, e.g. in a recent paper The Loss Surfaces of Multilayer Networks. In practice, what you find is that if you train a small network the final loss can display a good amount of variance - in some cases you get lucky and converge to a good place but in some cases you get trapped in one of the bad minima. On the other hand, if you train a large network you’ll start to find many different solutions, but the variance in the final achieved loss will be much smaller. In other words, all solutions are about equally as good, and rely less on the luck of random initialization.
- **The takeaway is that you should not be using smaller networks because you are afraid of overfitting. Instead, you should use as big of a neural network as your computational budget allows, and use other regularization techniques to control overfitting.**

# 3. Setting up the data and the model

## 3.1 Data preprocessing
There are three common forms of data preprocessing a data matrix $X$, where we will assume that $X$ is of size $[N x D]$ ($N$ is the number of data, $D$ is their dimensionality).

**Mean subtraction**: $X -= np.mean(X, axis = 0)$ or $X -= np.mean(X)$

**Normalization**: divide each dimension by its standard deviation, once it has been zero-centered: $X /= np.std(X, axis = 0)$ or preprocessing normalizes each dimension so that the min and max along the dimension is -1 and 1 respectively. 

**PCA and Whitening**: 
- $X -= np.mean(X, axis = 0)$ zero-center the data (important)
- $cov = np.dot(X.T, X) / X.shape[0]$ get the data covariance matrix
- $U,S,V = np.linalg.svd(cov)$
- $Xrot = np.dot(X, U)$
- $Xrotreduced = np.dot(X, U[:,:100])$ Xrot_reduced becomes $[N * 100]$
- $Xwhite = Xrot / np.sqrt(S + 1e-5)$

**In practice**: It is very important to zero-center the data, and it is common to see normalization of every pixel as well.

**Common pitfall**: The preprocessing is that any preprocessing statistics (e.g. the data mean) must only be computed on the training data, and then applied to the validation / test data.

## 3.2 Weight Initialization
**Pitfall**: all zero initialization. 
**Small random numbers**: It is common to initialize the weights of the neurons to small numbers and refer to doing so as symmetry breaking. $W = 0.01* np.random.randn(D,H)$
**Calibrating the variances with 1/sqrt(n)**: $w = \frac{np.random.randn(n) }{ \sqrt(n)}$. To normalize the variance of each neuron’s output to 1 by scaling its weight vector by the square root of its fan-in (i.e. its number of inputs). We want s to have the same variance as all of its inputs x.
- **Current recommendation**: $w = np.random.randn(n) * sqrt(2.0/n)$ (http://arxiv-web3.library.cornell.edu/abs/1502.01852)
**Sparse initialization**: ...
**Initialzing the biases**: It is possible and common to initialize the biases to be zero, since the asymmetry breaking is provided by the small random numbers in the weights. For ReLU non-linearities, some people like to use small constant value such as 0.01 for all biases because this ensures that all ReLU units fire in the beginning and therefore obtain and propagate some gradient. However, it is not clear if this provides a consistent improvement (in fact some results seem to indicate that this performs worse) and it is more common to simply use 0 bias initialization.

In practice: $w = np.random.randn(n) * sqrt(2.0/n)$

**Batch Normalization**: A recently developed technique by Ioffe and Szegedy called Batch Normalization alleviates a lot of headaches with properly initializing neural networks by explicitly forcing the activations throughout a network to take on a unit gaussian distribution at the beginning of the training. The core observation is that this is possible because normalization is a simple differentiable operation. In the implementation, applying this technique usually amounts to insert the BatchNorm layer immediately after fully connected layers (or convolutional layers, as we’ll soon see), and before non-linearities. We do not expand on this technique here because it is well described in the linked paper, but note that it has become a very common practice to use Batch Normalization in neural networks. In practice networks that use Batch Normalization are significantly more robust to bad initialization. Additionally, batch normalization can be interpreted as doing preprocessing at every layer of the network, but integrated into the network itself in a differentiable manner. Neat!
https://arxiv.org/abs/1502.03167


## 3.3 Regularization
**To prevent overfitting**:

- **L2 reg**: the appealing property of encouraging the network to use all of its inputs a little rather than some of its inputs a lot. During gradient descent parameter update, using the L2 regularization ultimately means that every weight is decayed linearly: W += -lambda * W towards zero.

- **L1 reg**: Neurons with L1 regularization end up using only a sparse subset of their most important inputs and become nearly invariant to the “noisy” inputs. In comparison, final weight vectors from L2 regularization are usually diffuse, small numbers. In practice, if you are not concerned with explicit feature selection, L2 regularization can be expected to give superior performance over L1.

- **Max norm constraints**:  To enforce an absolute upper bound on the magnitude of the weight vector for every neuron and use projected gradient descent to enforce the constraint. Its appealing properties is that network cannot “explode” even when the learning rates are set too high because the updates are always bounded.

- **Dropout**: While training, dropout is implemented by only keeping a neuron active with some probability p (a hyperparameter), or setting it to zero otherwise.
```
Train:
        H1 = np.maximum(0, np.dot(W1, X) + b1)
        U1 = np.random.rand(*H1.shape) < p # first dropout mask
        H1 *= U1 # drop!
        H2 = np.maximum(0, np.dot(W2, H1) + b2)
        U2 = np.random.rand(*H2.shape) < p # second dropout mask
        H2 *= U2 # drop!
        out = np.dot(W3, H2) + b3
Predict: 
        H1 = np.maximum(0, np.dot(W1, X) + b1) * p # NOTE: scale the activations
        H2 = np.maximum(0, np.dot(W2, H1) + b2) * p # NOTE: scale the activations
        out = np.dot(W3, H2) + b3

```
-> Predict function changed
-> Prefer inverted dropout: 

```
Train:
        H1 = np.maximum(0, np.dot(W1, X) + b1)
        U1 = (np.random.rand(*H1.shape) < p) / p # first dropout mask. Notice /p!
        H1 *= U1 # drop!
        H2 = np.maximum(0, np.dot(W2, H1) + b2)
        U2 = (np.random.rand(*H2.shape) < p) / p # second dropout mask. Notice /p!
        H2 *= U2 # drop!
        out = np.dot(W3, H2) + b3
Predict: 
        H1 = np.maximum(0, np.dot(W1, X) + b1) # no scaling necessary
        H2 = np.maximum(0, np.dot(W2, H1) + b2)
        out = np.dot(W3, H2) + b3

```
we show that the dropout regularizer is first-order equivalent to an L2 regularizer applied after scaling the features by an estimate of the inverse diagonal Fisher information matrix:
http://papers.nips.cc/paper/4882-dropout-training-as-adaptive-regularization.pdf

- **Theme of noise in forward pass**: ...

- **Bias regulazation**: It is not common to regularize the bias parameters because they do not interact with the data through multiplicative interactions, and therefore do not have the interpretation of controlling the influence of a data dimension on the final objective. However, in practical applications (and with proper data preprocessing) regularizing the bias rarely leads to significantly worse performance. This is likely because there are very few bias terms compared to all the weights, so the classifier can “afford to” use the biases if it needs them to obtain a better data loss.

- **Per-layer regularizarion**: It is not very common to regularize different layers to different amounts. 

- **In practice**:  It is most common to use a single, global L2 regularization strength that is cross-validated. It is also common to combine this with dropout applied after all layers. The value of p=0.5 is a reasonable default, but this can be tuned on validation data.


## 3.4 Loss function
- **Classification**: SVM, squared hingle loss, cross entropy loss.

- **Problem: Large number of classes**: When the set of labels is very large (e.g. words in English dictionary, or ImageNet which contains 22,000 categories), computing the full softmax probabilities becomes expensive. Hierarchical Softmax (https://arxiv.org/pdf/1310.4546.pdf)

- **Attribute classification**:  ... 
- **Regression**: Use L2/L1 norm between predicted value and true value. It is important to note that the L2 loss is much harder to optimize than a more stable loss such as Softmax. L2 loss is less robust because outliers can introduce huge gradients. When faced with a regression problem, first consider if it is absolutely inadequate to quantize the output into bins.

- **Structured prediction**: ...

# 4. Learning

## 4.1 Gradient Checks
- **Use the centered formula**: $\frac{df(x)}{dx} = \frac{f(x+h) - f(x-h)}{2h}$
- **Use relative error for the comparison**: $\frac{|f_a' - f_n'|}{max(|f_a'|,|f_n'|)}$. 

+ relative error > 1e-2 usually means the gradient is probably wrong
+ 1e-2 > relative error > 1e-4 should make you feel uncomfortable
+ 1e-4 > relative error is usually okay for objectives with kinks. But if there are no kinks (e.g. use of tanh nonlinearities and softmax), then 1e-4 is too high.
+ 1e-7 and less you should be happy.

- **Use double precision**

- **Stick around active range of floating point**: always print the raw numerical/analytic gradient, and make sure that the numbers you are comparing are not extremely small. 

- **Kinks in the objective**: Kinks refer to non-differentiable parts of an objective function. This case can be very common. 

- **Use only few datapoints**: One fix to the above problem of kinks is to use fewer datapoints, since loss functions that contain kinks (e.g. due to use of ReLUs or margin losses etc.) will have fewer kinks with fewer datapoints, so it is less likely for you to cross one when you perform the finite different approximation. Moreover, if your gradcheck for only ~2 or 3 datapoints then you would almost certainly gradcheck for an entire batch. Using very few datapoints also makes your gradient check faster and more efficient.

- **Be careful with the step size h**: It is not necessarily the case that smaller is better, because when h is much smaller, you may start running into numerical precision problems.

- **Gradcheck during a “characteristic” mode of operation**: ...
- **Don’t let the regularization overwhelm the data**: it is recommended to turn off regularization and check the data loss alone first, and then the regularization term second and independently. One way to perform the latter is to hack the code to remove the data loss contribution. Another way is to increase the regularization strength so as to ensure that its effect is non-negligible in the gradient check, and that an incorrect implementation would be spotted.

- **Remember to turn off dropout/augmentations**: When performing gradient check, remember to turn off any non-deterministic effects in the network, such as dropout, random data augmentations, etc.

- **Check only few dimensions**: One issue to be careful with is to make sure to gradient check a few dimensions for every separate parameter.

## 4.2 Before learning: santy check Tips/Tricks
- **Look for correct loss at chance performance**: Make sure you’re getting the loss you expect when you initialize with small parameters. It’s best to first check the data loss alone (so set regularization strength to zero). For example, for CIFAR-10 with a Softmax classifier we would expect the initial loss to be 2.302, because we expect a diffuse probability of 0.1 for each class (since there are 10 classes), and Softmax loss is the negative log probability of the correct class so: -ln(0.1) = 2.302. For The Weston Watkins SVM, we expect all desired margins to be violated (since all scores are approximately zero), and hence expect a loss of 9 (since margin is 1 for each wrong class). If you’re not seeing these losses there might be issue with initialization.

- As a second sanity check, increasing the regularization strength should increase the loss.

- **Overfit a tiny subset of data**: Before training on the full dataset try to train on a tiny portion (e.g. 20 examples) of your data and make sure you can achieve zero cost. For this experiment it’s also best to set regularization to zero, otherwise this can prevent you from getting zero cost. Unless you pass this sanity check with a small dataset it is not worth proceeding to the full dataset. Note that it may happen that you can overfit very small dataset but still have an incorrect implementation. For instance, if your datapoints’ features are random due to some bug, then it will be possible to overfit your small training set but you will never notice any generalization when you fold it your full dataset.

## 4.3 Babysitting the learning process

### 4.3.1 Loss function
- The amount of “wiggle” in the loss is related to the batch size. When the batch size is 1, the wiggle will be relatively high. When the batch size is the full dataset, the wiggle will be minimal because every gradient update should be improving the loss function monotonically (unless the learning rate is set too high)
### 4.3.2 Train/Val accuracy
- The gap between the training and validation accuracy indicates the amount of overfitting.

### 4.3.3 Ratio of weights/updates
```
    # assume parameter vector W and its gradient vector dW
    param_scale = np.linalg.norm(W.ravel())
    update = -learning_rate*dW # simple SGD update
    update_scale = np.linalg.norm(update.ravel())
    W += update # the actual update
    print update_scale / param_scale # want ~1e-3
```
Around 1e-3 is good. If it is lower than this then the learning rate might be too low. If it is higher then the learning rate is likely too high.

### 4.3.4 Activation / Gradient distributions per layer
An incorrect initialization can slow down or even completely stall the learning process. Luckily, this issue can be diagnosed relatively easily. One way to do so is to plot activation/gradient histograms for all layers of the network. Intuitively, it is not a good sign to see any strange distributions - e.g. with tanh neurons we would like to see a distribution of neuron activations between the full range of [-1,1], instead of seeing all neurons outputting zero, or all neurons being completely saturated at either -1 or 1.

### 4.3.5 First layer visualizations


## 4.4 Parameter updates

### 4.4.1 SGD and bells and whistles
- **Vanilla update**: $x += - learning_rate * dx$
- **Momentum update**: 
$v = mu * v - learning_rate * dx$ # integrate velocity
$x += v$ # integrate position

- **Nesterov Momentum**: 
```
x_ahead = x + mu * v
# evaluate dx_ahead (the gradient at x_ahead instead of at x)
v = mu * v - learning_rate * dx_ahead
x += v
```
```
v_prev = v # back this up
v = mu * v - learning_rate * dx # velocity update stays the same
x += -mu * v_prev + (1 + mu) * v # position update changes form
```
### 4.4.2 Annealing the learning rate
There are three common types of implementing the learning rate decay:
- **Step decay**: Reduce the learning rate by some factor every few epochs
- **Exponential decay**: has the mathematical form $\alpha=\alpha_0e^{−kt}$, where $\alpha_0$, $k$ are hyperparameters and $t$ is the iteration number (but you can also use units of epochs).
- **1/t** decay: $\alpha = \frac{\alppha_0}{1 + kt}$, t is the iteration number, $\alpha_0$, k are constants.

### 4.4.3 Second order methods
...
In practice, SGD variants based on (Nesterov’s) momentum are more standard because they are simpler and scale more easily.
### 4.4.4 Per-parameter adaptive learning rate methods

- **Adagrad**: 
```
    # Assume the gradient dx and parameter vector x
    cache += dx**2
    x += - learning_rate * dx / (np.sqrt(cache) + eps)
```
The weights that receive high gradients will have their effective learning rate reduced, while weights that receive small or infrequent updates will have their effective learning rate increased. **in case of Deep Learning, the monotonic learning rate usually proves too aggressive and stops learning too early.**

- **RMSprop**: 
```
    cache = decay_rate * cache + (1 - decay_rate) * dx**2
    x += - learning_rate * dx / (np.sqrt(cache) + eps)
```
RMSProp still modulates the learning rate of each weight based on the magnitudes of its gradients, which has a beneficial equalizing effect, but unlike Adagrad the updates do not get monotonically smaller.

- **Adam**: 
```
    m = beta1*m + (1-beta1)*dx
    v = beta2*v + (1-beta2)*(dx**2)
    x += - learning_rate * m / (np.sqrt(v) + eps)
```
The full Adam update: includes a bias correction mechanism, which compensates for the fact that in the first few time steps the vectors m,v are both initialized and therefore biased at zero, before they fully “warm up”.
```
    # t is your iteration counter going from 1 to infinity
    m = beta1*m + (1-beta1)*dx
    mt = m / (1-beta1**t)
    v = beta2*v + (1-beta2)*(dx**2)
    vt = v / (1-beta2**t)
    x += - learning_rate * mt / (np.sqrt(vt) + eps)
```


**In practice Adam is currently recommended as the default algorithm to use, and often works slightly better than RMSProp. However, it is often also worth trying SGD+Nesterov Momentum as an alternative.**




## 4.5 Hyperparameter optimization



## 4.6 Evaluation




# Related Question/notes/...
- **how to control kernel differences things to learn (maximum n vectors which have dim=n and independent)**
- **size of kernel selection**
- NIN: https://arxiv.org/abs/1312.4400
- https://arxiv.org/abs/1511.07122
- https://arxiv.org/abs/1412.6806
- http://www.deeplearningbook.org/
- https://arxiv.org/abs/1312.6184
- https://arxiv.org/abs/1412.6550
- https://arxiv.org/abs/1502.03167
- http://web.stanford.edu/~hastie/Papers/B67.2%20%282005%29%20301-320%20Zou%20&%20Hastie.pdf
- http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf